In [8]:
from obspy.core import Stream, read, UTCDateTime
import matplotlib.pyplot as plt
import numpy as np
import h5py

stas = np.genfromtxt('/hdd/Ridgecrest/summer23/GNSS_stas.txt', usecols = [2], dtype = str)
chans = ['e', 'n', 'u']
eq_dates = np.genfromtxt('/hdd/Ridgecrest/summer23/GNNS_dates_w_eqs_only.txt', dtype = str)
ns = np.arange(0,680,1)

project = 'summer23'
project_dir = '/hdd/Ridgecrest/' + project + '/'
split_mseed_path = project_dir + 'eq_days_split_mseeds/' # Where the split data we're loading is

small_test = False # Just checking to see how the code runs? If so, True

if small_test:
    
    stas = ['BEPK', 'CCCC']
    dates = ['20190705', '20190712']

data_list = []
k = -1

sta_list = []
date_list = []
starttime_list = []
endtime_list = []
n_list = []

In [9]:
for sta in stas:
        
    for date in dates:
        
        print(sta + '_' + date)

        for n in ns:
        
            try:
                
                stN = read(split_mseed_path + sta + '/n_split_mseeds/' + date + '/' + sta + '.n.' + date + '.' + str(n) + '.mseed')
                stE = read(split_mseed_path + sta + '/e_split_mseeds/' + date + '/' + sta + '.e.' + date + '.' + str(n) + '.mseed')
                stZ = read(split_mseed_path + sta + '/u_split_mseeds/' + date + '/' + sta + '.u.' + date + '.' + str(n) + '.mseed')
                
                N_data = stN[0].data
                E_data = stE[0].data
                Z_data = stZ[0].data
                
                comb_data = np.append(N_data, E_data)
                comb_data = np.append(comb_data, Z_data) # Order: N, E, Z

                data_list.append(comb_data)
                
                starttime = stN[0].stats.starttime
                endtime = stN[0].stats.endtime
                
                sta_list.append(sta)
                date_list.append(date)
                starttime_list.append(str(starttime))
                endtime_list.append(str(endtime))
                n_list.append(n)
                
                # Plot
                
                # print(k)
                # if k == 678:
        
                #     plt.figure(figsize=(10,4), dpi=300)
                #     plt.plot(comb_data,label='Real data')
                #     # plt.plot(one_line_noise,label='Noise')
                #     # plt.plot(noisy_data-0.2,label='Noisy data')
                #     # plt.legend(loc = 'upper right')
                    
                #     plt.axvline(127, color = 'black') # last sample of N. E starts at 256
                #     plt.axvline(255, color = 'black') # last sample of E. Z starts at 512
                    
                #     plt.text(25, -0.35, 'N', fontsize = 25, fontweight = 'bold')
                #     plt.text(100, -0.35, 'E', fontsize = 25, fontweight = 'bold')
                #     plt.text(175, -0.35, 'Z', fontsize = 25, fontweight = 'bold')
                    
                #     # plt.show()
                #     plt.savefig('plot.png', format = 'PNG')
                #     plt.close()
                
                k += 1
                
            except:
                
                print('Missing station, channel, or date')



BEPK_20190705
BEPK_20190712
CCCC_20190705
CCCC_20190712


In [10]:
data_array = np.array(data_list)
print('Data array shape:')
print(data_array.shape) # should be something by 384

info_array = np.column_stack((sta_array, date_array, starttime_array, endtime_array, n_array))
print('Info array shape:')
print(info_array.shape)

h5f = h5py.File(project_dir + 'realdata_data.hdf5', 'w') 
h5f.create_dataset('realdata_data', data = data_array)
h5f.close()

np.save(project_dir + 'realdata_info.npy', info_array) 

Data array shape:
(2720, 384)
Info array shape:
(2720, 5)
